In [1]:
import tensorflow as tf
import os
import cv2
import pickle
from tqdm import tqdm
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [2]:
def extract_images_from_folder(root_folder, target_size=(256, 256)):
    image_data = []
    labels = []
    label_to_index = {}  # Dictionary to map label names to numerical indices
    index = 0

    for subdir, dirs, files in os.walk(root_folder):
        for file in tqdm(files):
            filepath = os.path.join(subdir, file)
            img = cv2.imread(filepath)
            img = cv2.resize(img, target_size)

            label = os.path.basename(subdir)
            if label not in label_to_index:
                label_to_index[label] = index
                index += 1

            image_data.append(img)
            labels.append(label_to_index[label])

    image_data = np.array(image_data)
    labels = np.array(labels)

    return image_data, labels, label_to_index

In [4]:
root_folder = r"C:\Users\vinit\Downloads\plantdisease\Train"
x_train,y_train,label_reverse_train = extract_images_from_folder(root_folder)

0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|████████████████████████████████████████████████████████████████████████████████| 434/434 [01:10<00:00,  6.19it/s]


In [5]:
root_folder = r"C:\Users\vinit\Downloads\plantdisease\Validation"
x_val,y_val,label_reverse_val = extract_images_from_folder(root_folder)

0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:03<00:00,  5.75it/s]


In [6]:
root_folder = r"C:\Users\vinit\Downloads\plantdisease\Test"
x_test,y_test,label_reverse_test = extract_images_from_folder(root_folder)

0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:09<00:00,  5.49it/s]


In [7]:
x_train = x_train / 255.0
x_test = x_test / 255.0
x_val = x_val / 255.0

In [8]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_reverse_train), activation='softmax')
])

In [9]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
history = model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))

Epoch 1/10
42/42 [==============================] - 109s 2s/step - loss: 0.9845 - accuracy: 0.5703 - val_loss: 0.7409 - val_accuracy: 0.7167
Epoch 2/10
42/42 [==============================] - 85s 2s/step - loss: 0.4443 - accuracy: 0.8411 - val_loss: 0.4228 - val_accuracy: 0.8500
Epoch 3/10
42/42 [==============================] - 86s 2s/step - loss: 0.3293 - accuracy: 0.8888 - val_loss: 0.4586 - val_accuracy: 0.8167
Epoch 4/10
42/42 [==============================] - 84s 2s/step - loss: 0.2090 - accuracy: 0.9402 - val_loss: 0.2748 - val_accuracy: 0.9000
Epoch 5/10
42/42 [==============================] - 82s 2s/step - loss: 0.1855 - accuracy: 0.9402 - val_loss: 0.3557 - val_accuracy: 0.8500
Epoch 6/10
42/42 [==============================] - 82s 2s/step - loss: 0.1490 - accuracy: 0.9523 - val_loss: 0.3284 - val_accuracy: 0.8833
Epoch 7/10
42/42 [==============================] - 82s 2s/step - loss: 0.2571 - accuracy: 0.9115 - val_loss: 0.4165 - val_accuracy: 0.8333
Epoch 8/10
42/42 [=

In [11]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test accuracy:', test_acc)

5/5 [==============================] - 3s 533ms/step - loss: 0.5173 - accuracy: 0.8600
Test accuracy: 0.8600000143051147


In [ ]:
#https://www.kaggle.com/datasets/rashikrahmanpritom/plant-disease-recognition-dataset

In [13]:
y_pred = model.predict(x_test)

5/5 [==============================] - 3s 486ms/step


In [22]:
y_pred[60]

array([4.2224565e-05, 9.9992311e-01, 3.4668479e-05], dtype=float32)

In [23]:
y_test[60]

1